In [ ]:
"""
The MIT License (MIT)
Copyright (c) 2021 NVIDIA
Permission is hereby granted, free of charge, to any person obtaining a copy of
this software and associated documentation files (the "Software"), to deal in
the Software without restriction, including without limitation the rights to
use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of
the Software, and to permit persons to whom the Software is furnished to do so,
subject to the following conditions:
The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.
THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS
FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR
COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER
IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN
CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.
"""


This code example demonstrates how to use a convolutional neural network to solve an image classification problem, using the CIFAR-10 dataset. More context for this code example can be found in video 4.5 "Programming Example: Image Classification Using CNN with PyTorch" in the video series "Learning Deep Learning: From Perceptron to Large Language Models" by Magnus Ekman (Video ISBN-13: 9780138177614). This is notebook 2 of 3 for that example.


The first code snippet shows the initialization code for our CNN program.

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
import numpy as np
from utilities import train_model

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
EPOCHS = 32
BATCH_SIZE = 32

# Load training dataset into a single batch to compute mean and stddev.
transform = transforms.Compose([transforms.ToTensor()])
trainset = CIFAR10(root='./pt_data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=len(trainset), shuffle=False)
data = next(iter(trainloader))
mean = data[0].mean()
stddev = data[0].std()

# Load and standardize training and test dataset.
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize(mean, stddev)])

trainset = CIFAR10(root='./pt_data', train=True, download=True, transform=transform)
testset = CIFAR10(root='./pt_data', train=False, download=True, transform=transform)


The actual model is created by the next code snippet, which declares a Sequential model. We are now working with a 2D convolutional layer, so there is no need to start with a Flatten layer because the dimensions of the input image already match the required dimension of the first layer. We tell the layer that the input image has 3 channels and we want the layer to output 64 channels. We also state that we want a kernel size of 5×5, a stride of 2x2. Finally, we state that we want to pad the image with 2 pixels on each side, which in combination with the 5x5 kernel size and 2x2 stride will result in the output resolution being half of the resolution of the input image. That is, the output of the layer will be 16x16 neurons per channel. We specify the neuron type as ReLU because that has been shown to be a good activation function.

The next convolutional layer is similar but with a smaller kernel size. The number of input channels (64) must match the number of output channels from the previous layer. The number of neurons per channel is implicitly defined as 8×8 because the previous layer was 16×16 outputs per channel, and we choose a stride of 2x2 for this layer as well. We set padding to 1 since our kernel size is 3.

Before we can add the fully connected (Linear) layer, we need to flatten (convert from three dimensions to a single dimension) the outputs from the second convolutional layer. We compute the number of inputs for this layer as 64x8x8 (the previous convolutional layer was 64 channels and 8x8 neurons per channel). We use softmax activation for the fully connected layer so we can interpret the one-hot encoded outputs as probabilities.

We use Kaiming (He) initialization for the weights in the convolutional layers and Xavier (Glorot) for the fully connected layer.

We finally select the categorical_crossentropy loss function and use the Adam optimizer in our call to compile. We then print out a description of the network with a call to model.summary().


In [ ]:
# Model with 2 convolutional and 1 fully-connected layer.
model = nn.Sequential(
    nn.Conv2d(3, 64, 5, stride=2, padding=2), # Output is 64x16x16.
    nn.ReLU(),
    nn.Conv2d(64, 64, 3, stride=2, padding=1), # Output is 64x8x8.
    nn.ReLU(),
    nn.Flatten(),
    nn.Linear(64 * 8 * 8, 10)
)

# Retrieve layers for custom weight initialization.
layers = next(model.modules())
conv_layer0 = layers[0]
conv_layer1 = layers[2]
output_layer = layers[5]

# Kaiming (He) initialization.
nn.init.kaiming_normal_(conv_layer0.weight)
nn.init.constant_(conv_layer0.bias, 0.0)
nn.init.kaiming_normal_(conv_layer1.weight)
nn.init.constant_(conv_layer1.bias, 0.0)

# Xavier (Glorot) initialization.
nn.init.xavier_uniform_(output_layer.weight)
nn.init.constant_(output_layer.bias, 0.0)

# Loss function and optimizer
optimizer = torch.optim.Adam(model.parameters())
loss_function = nn.CrossEntropyLoss()


We are now ready to train the network by calling the same train_model() function that was used in the v3_15_regression example, but this time we give the argument "acc" because we work on a classification problem and therefore want the function to compute accuracy instead of mean absolute error (mae).


In [ ]:
# Train the model.
train_model(model, device, EPOCHS, BATCH_SIZE, trainset, testset,
            optimizer, loss_function, 'acc')
